## Accuracy

In [5]:
!nvidia-smi

Thu Jun 27 07:58:30 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   34C    P0    73W / 300W |   1446MiB / 16130MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:1C:00.0 Off |                    0 |
| N/A   30C    P0    55W / 300W |   1178MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="3" 

In [3]:
import sys
sys.path.append("/trinity/home/y.gusak/catalyst")
from catalyst.dl.callbacks import Callback, AccuracyCallback
from catalyst.dl.experiments import SupervisedRunner

from collections import OrderedDict, defaultdict

import torch
import numpy as np

ImportError: cannot import name 'Callback'

In [3]:
loaders = OrderedDict()

In [4]:
import sys
sys.path.append('../')
import dataloaders

import torchvision.datasets as datasets
from torchvision import transforms

# DATA_ROOT = "/workspace/raid/data/datasets"
DATA_ROOT = "/gpfs/gpfs0/e.ponomarev/"
dataset_name = 'imagenet'

bs = 128
num_workers = 32

if dataset_name == 'cifar10':
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    DATA_PATH = "{}/cifar10".format(DATA_ROOT)

    loaders["valid"] = torch.utils.data.DataLoader(
        datasets.CIFAR10(root=DATA_PATH, train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]),  download = True),
        batch_size=bs, shuffle=False,
        num_workers=num_workers, pin_memory=True)
elif dataset_name == 'imagenet':
    loaders["valid"] = dataloaders.get_loader(batch_size=bs,
                                        data_name = 'imagenet',
                                        data_root = DATA_ROOT,
                                        num_workers = num_workers, 
                                        pin_memory = True)['val']
    loaders["train"] = dataloaders.get_loader(batch_size=bs,
                                    data_name = 'imagenet',
                                    data_root = DATA_ROOT,
                                    num_workers = num_workers, 
                                    pin_memory = True)['train']

Building imagenet data loader with 32 workers
Building imagenet data loader with 32 workers


In [5]:
class ValCallback(Callback):
    
    
    def __init__(self, out_dir=None, out_prefix=None):
        self.out_dir = out_dir
        self.out_prefix = out_prefix
        self.predictions = defaultdict(lambda: [])
        self._keys_from_state = ["out_dir", "out_prefix"]

    def on_stage_start(self, state):
        for key in self._keys_from_state:
            value = getattr(state, key, None)
            if value is not None:
                setattr(self, key, value)
        # assert self.out_prefix is not None
        if self.out_dir is not None:
            self.out_prefix = str(self.out_dir) + "/" + str(self.out_prefix)
        if self.out_prefix is not None:
            os.makedirs(os.path.dirname(self.out_prefix), exist_ok=True)

    def on_loader_start(self, state):
        self.predictions = defaultdict(lambda: [])
        self.inputs = defaultdict(lambda: [])

    def on_batch_end(self, state):
        dct = state.output
        dct = {key: value.detach().cpu().numpy() for key, value in dct.items()}
        for key, value in dct.items():
            self.predictions[key].append(value)
            
        dct = state.input
        dct = {key: value.detach().cpu().numpy() for key, value in dct.items()}
        for key, value in dct.items():
            self.inputs[key].append(value)

    def on_loader_end(self, state):
        from catalyst.dl.metrics import accuracy
        self.predictions = {
            key: np.concatenate(value, axis=0)
            for key, value in self.predictions.items()
        }
        self.inputs = {
            key: np.concatenate(value, axis=0)
            for key, value in self.inputs.items()
        }
        y_true = torch.tensor(self.inputs['targets'])
        y_pred = torch.tensor(self.predictions['logits'])
        self.accuracy_score = torch.stack(accuracy(y_pred,y_true,topk=(1,5))).numpy().astype('float32')
        self.accuracy_score = np.squeeze(self.accuracy_score)

In [6]:
import sys
sys.path.append('../')

from model_utils import load_model
from torchvision.models import resnet34

# MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'
# MODEL_NAME = 'resnet18_imagenet'
MODEL_NAME = 'resnet34_imagenet'

# MODEL_NAME = 'faster_rcnn_vgg16
# MODEL_NAME = 'faster_rcnn_resnet50'


# # Uncomment if MODEL_NAME = 'faster_rcnn_resnet50'
# sys.path.append('/workspace/home/jgusak/maxvol_objects/facebook_frcnn/')
# import maskrcnn_benchmark

# model = load_model(MODEL_NAME)

model = resnet34(pretrained = True)

In [ ]:
lr = 0.002
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-6,lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    callbacks=[
        AccuracyCallback(accuracy_args = [1,5]), # top (1,5) accuracy  on batch
#         ValCallback()
    ],
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir='./logs',
    num_epochs=15,
    verbose=True,
    valid_loader='valid'
    
)



0/15 * Epoch (valid): 100% 391/391 [01:09<00:00,  5.13it/s, _timers/_fps=770.842, accuracy01=46.250, accuracy05=86.250, loss=1.938]   
0/15 * Epoch (train):   2% 189/10010 [00:59<29:18,  5.58it/s, _timers/_fps=3329.246, accuracy01=4.688, accuracy05=15.625, loss=5.538]  

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



0/15 * Epoch (train):  22% 2232/10010 [11:21<27:35,  4.70it/s, _timers/_fps=4496.783, accuracy01=32.812, accuracy05=56.250, loss=3.249]  

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



0/15 * Epoch (train):  52% 5215/10010 [27:03<14:36,  5.47it/s, _timers/_fps=3250.650, accuracy01=38.281, accuracy05=57.812, loss=3.081]  

...

7/15 * Epoch (train):  62% 6161/10010 [19:56<11:31,  5.57it/s, _timers/_fps=3475.073, accuracy01=66.406, accuracy05=85.938, loss=1.437]  

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



7/15 * Epoch (train):  64% 6424/10010 [20:46<10:38,  5.61it/s, _timers/_fps=3470.692, accuracy01=54.688, accuracy05=75.781, loss=1.921]

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



7/15 * Epoch (train):  68% 6775/10010 [21:54<09:59,  5.40it/s, _timers/_fps=2923.592, accuracy01=62.500, accuracy05=81.250, loss=1.532]

In [8]:
scores0 = run(model)

0/1 * Epoch (infer): 100% 391/391 [01:09<00:00,  6.08it/s, _timers/_fps=684.177, accuracy01=46.250, accuracy03=80.000, accuracy05=86.250]  


In [9]:
scores0

array([73.314   , 87.728004, 91.420006], dtype=float32)